<a href="https://colab.research.google.com/github/Afsoon-Ghazanfari/Salary-Predictor/blob/main/cleaning_outliers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#connection to Google Drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


In [3]:
#creating dataframe from Drive
df = pd.read_csv('/content/drive/My Drive/RediSchool-salarypredictor/survey_results_public.csv')
df.head()

,ResponseId,Q120,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,ProfessionalTech,Industry,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I agree,None of these,18-24 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I agree,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Boots...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;Friend or fam...,Formal documentation provided by the owner of ...,...,1-2 times a week,10+ times a week,Never,15-30 minutes a day,15-30 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Appropriate in length,Easy,285000.0
2,3,I agree,I am a developer by profession,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Professional development or self-paced l...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Formal documentation provided by the owner of ...,...,6-10 times a week,6-10 times a week,3-5 times a week,30-60 minutes a day,30-60 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Appropriate in length,Easy,250000.0
3,4,I agree,I am a developer by profession,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Colleague;Friend or family member;Other online...,Formal documentation provided by the owner of ...,...,1-2 times a week,10+ times a week,1-2 times a week,15-30 minutes a day,30-60 minutes a day,Automated testing;Continuous integration (CI) ...,NaN,Appropriate in length,Easy,156000.0
4,5,I agree,I am a developer by profession,25-34 years old,"Employed, full-time;Independent contractor, fr...",Remote,Hobby;Contribute to open-source projects;Profe...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Formal documentation provided by the owner of ...,...,1-2 times a week,1-2 times a week,3-5 times a week,60-120 minutes a day,30-60 minutes a day,Microservices;Automated testing;Observability ...,Other,Appropriate in length,Neither easy nor difficult,23456.0


In [4]:
df.columns

Index(['ResponseId', 'Q120', 'MainBranch', 'Age', 'Employment', 'RemoteWork',
       'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline',
       'LearnCodeCoursesCert', 'YearsCode', 'YearsCodePro', 'DevType',
       'OrgSize', 'PurchaseInfluence', 'TechList', 'BuyNewTool', 'Country',
       'Currency', 'CompTotal', 'LanguageHaveWorkedWith',
       'LanguageWantToWorkWith', 'DatabaseHaveWorkedWith',
       'DatabaseWantToWorkWith', 'PlatformHaveWorkedWith',
       'PlatformWantToWorkWith', 'WebframeHaveWorkedWith',
       'WebframeWantToWorkWith', 'MiscTechHaveWorkedWith',
       'MiscTechWantToWorkWith', 'ToolsTechHaveWorkedWith',
       'ToolsTechWantToWorkWith', 'NEWCollabToolsHaveWorkedWith',
       'NEWCollabToolsWantToWorkWith', 'OpSysPersonal use',
       'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith',
       'OfficeStackAsyncWantToWorkWith', 'OfficeStackSyncHaveWorkedWith',
       'OfficeStackSyncWantToWorkWith', 'AISearchHaveWorkedWith',
       'AISearchWan

In [5]:
#filtering with country
df=df[df.Country=='Germany']

In [6]:
len(df)

7328

In [47]:
#df_new=df[['MainBranch','Employment','RemoteWork','YearsCodePro','DevType','OrgSize','CompTotal','Country']]
df_new=df[['Age','RemoteWork','YearsCodePro','EdLevel','OrgSize','Employment','DevType','LanguageHaveWorkedWith','ToolsTechHaveWorkedWith','ProfessionalTech','CompTotal']]

In [48]:
df_new.head()

,Age,RemoteWork,YearsCodePro,EdLevel,OrgSize,Employment,DevType,LanguageHaveWorkedWith,ToolsTechHaveWorkedWith,ProfessionalTech,CompTotal
15,35-44 years old,"Hybrid (some remote, some in-person)",10,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",100 to 499 employees,"Employed, full-time","Developer, back-end",Bash/Shell (all shells);Dart;Go;Haskell;HTML/C...,Ansible;Ant;Docker;Gradle;Kubernetes;Make;Mave...,Microservices;Automated testing;Observability ...,100000.0
19,35-44 years old,"Hybrid (some remote, some in-person)",14,Some college/university study without earning ...,20 to 99 employees,"Employed, full-time","Developer, full-stack",Java;JavaScript;TypeScript,Docker;Gradle;npm;Podman;Terraform;Vite,Observability tools;Continuous integration (CI...,70044.0
21,25-34 years old,Remote,5,"Professional degree (JD, MD, Ph.D, Ed.D, etc.)",500 to 999 employees,"Employed, full-time",Data scientist or machine learning specialist,Bash/Shell (all shells);Delphi;HTML/CSS;PHP;Po...,APT;LLVM's Clang;Maven (build tool);Pip,None of these,80000.0
23,25-34 years old,Remote,10,"Professional degree (JD, MD, Ph.D, Ed.D, etc.)",500 to 999 employees,"Employed, full-time",Data or business analyst,Bash/Shell (all shells);C++;PowerShell;Python,Docker,NaN,65000.0
32,25-34 years old,"Hybrid (some remote, some in-person)",7,"Secondary school (e.g. American high school, G...",10 to 19 employees,"Employed, full-time","Developer, back-end",Java;JavaScript;Lua;Python;SQL;TypeScript,Cargo;Dagger;Docker;Gradle;Make;Maven (build t...,DevOps function;Microservices;Observability to...,50000.0


In [49]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7328 entries, 15 to 89146
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      7328 non-null   object 
 1   RemoteWork               6234 non-null   object 
 2   YearsCodePro             5355 non-null   object 
 3   EdLevel                  7328 non-null   object 
 4   OrgSize                  5267 non-null   object 
 5   Employment               7321 non-null   object 
 6   DevType                  6328 non-null   object 
 7   LanguageHaveWorkedWith   7298 non-null   object 
 8   ToolsTechHaveWorkedWith  6727 non-null   object 
 9   ProfessionalTech         3385 non-null   object 
 10  CompTotal                3981 non-null   float64
dtypes: float64(1), object(10)
memory usage: 687.0+ KB


In [50]:
# count the number of missing data and then proceed
missing_data = df_new.isnull()
print("Below Results shows there are no missing data in first 5 samples (or rows) ")
display(missing_data.head())

Below Results shows there are no missing data in first 5 samples (or rows) 


,Age,RemoteWork,YearsCodePro,EdLevel,OrgSize,Employment,DevType,LanguageHaveWorkedWith,ToolsTechHaveWorkedWith,ProfessionalTech,CompTotal
15,False,False,False,False,False,False,False,False,False,False,False
19,False,False,False,False,False,False,False,False,False,False,False
21,False,False,False,False,False,False,False,False,False,False,False
23,False,False,False,False,False,False,False,False,False,True,False
32,False,False,False,False,False,False,False,False,False,False,False


In [51]:
#missing data count for each column
for column in missing_data.columns.values.tolist():
    print(column)
    display(missing_data[column].value_counts())
    print("")

Age


False    7328
Name: Age, dtype: int64


RemoteWork


False    6234
True     1094
Name: RemoteWork, dtype: int64


YearsCodePro


...

False    6727
True      601
Name: ToolsTechHaveWorkedWith, dtype: int64


ProfessionalTech


True     3943
False    3385
Name: ProfessionalTech, dtype: int64


CompTotal


False    3981
True     3347
Name: CompTotal, dtype: int64

In [52]:
#create a box plot
import plotly.express as px

fig = px.box(df_new, y='CompTotal')

fig.show()

In [53]:
#create new dataframe without null values in CompTotal column
df_new2=df_new[~df_new.CompTotal.isnull()]

In [54]:
#create IQR for outlier
IQR = df_new2['CompTotal'].quantile(0.75) - df_new2['CompTotal'].quantile(0.25)
IQR

35000.0

In [55]:
#finding higher and lower outlier
Higher_outlier = df_new2['CompTotal'].quantile(0.75) + (IQR * 1.5)
lower_outlier = df_new2['CompTotal'].quantile(0.25) - (IQR * 1.5)
print(Higher_outlier)
print(lower_outlier)

142500.0
2500.0


In [56]:
print("Higher")
print(df_new2.loc[df_new2['CompTotal'] >= Higher_outlier, 'CompTotal'])
print("Lower")
print(df_new2.loc[df_new2['CompTotal']<lower_outlier+10000,'CompTotal'])


Higher
103      150000.0
539      190000.0
915      220000.0
1025     160000.0
2023     170000.0
           ...   
82357    143000.0
84582    204000.0
85407    250000.0
86242    300000.0
87383    150000.0
Name: CompTotal, Length: 180, dtype: float64
Lower
494       6200.0
755       4000.0
929       3125.0
1150      8000.0
1429      5335.0
          ...   
82569    10680.0
85405     9000.0
85618     6240.0
88379     5400.0
89146     1500.0
Name: CompTotal, Length: 85, dtype: float64


In [57]:
#create and plot values without higher outlier
df_new3=df_new2[df_new2['CompTotal'] <= Higher_outlier-10000]
fig = px.box(df_new3, y='CompTotal')

fig.show()

In [58]:
len(df_new3)

3764

In [59]:
#create and plot values without outliers
df_new4=df_new3[df_new3['CompTotal'] >= lower_outlier+10000]
fig = px.box(df_new4, y='CompTotal')

fig.show()

In [60]:
# count the number of missing data and then proceed
missing_data = df_new4.isnull()
print("Below Results shows there are no missing data in first 5 samples (or rows) ")
display(missing_data.head())
for column in missing_data.columns.values.tolist():
    print(column)
    display(missing_data[column].value_counts())
    print("")

Below Results shows there are no missing data in first 5 samples (or rows) 


,Age,RemoteWork,YearsCodePro,EdLevel,OrgSize,Employment,DevType,LanguageHaveWorkedWith,ToolsTechHaveWorkedWith,ProfessionalTech,CompTotal
15,False,False,False,False,False,False,False,False,False,False,False
19,False,False,False,False,False,False,False,False,False,False,False
21,False,False,False,False,False,False,False,False,False,False,False
23,False,False,False,False,False,False,False,False,False,True,False
32,False,False,False,False,False,False,False,False,False,False,False


Age


False    3679
Name: Age, dtype: int64


RemoteWork


...

False    3439
True      240
Name: ToolsTechHaveWorkedWith, dtype: int64


ProfessionalTech


False    2464
True     1215
Name: ProfessionalTech, dtype: int64


CompTotal


False    3679
Name: CompTotal, dtype: int64

In [61]:
#dropping nan values
df_new4=df_new4.dropna()

In [62]:
len(df_new4)

2345

In [64]:
df

,ResponseId,Q120,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,ProfessionalTech,Industry,SurveyLength,SurveyEase,ConvertedCompYearly
15,16,I agree,I am a developer by profession,35-44 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Contribute to open-source projects;Profe...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Other online resources ...,Formal documentation provided by the owner of ...,...,1-2 times a week,1-2 times a week,1-2 times a week,15-30 minutes a day,15-30 minutes a day,Microservices;Automated testing;Observability ...,"Manufacturing, Transportation, or Supply Chain",Too long,Neither easy nor difficult,107090.0
19,20,I agree,I am a developer by profession,35-44 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Professional development or self-paced l...,Some college/university study without earning ...,Online Courses or Certification;On the job tra...,Formal documentation provided by the owner of ...,...,1-2 times a week,3-5 times a week,1-2 times a week,15-30 minutes a day,15-30 minutes a day,Observability tools;Continuous integration (CI...,"Information Services, IT, Software Development...",Appropriate in length,Easy,75010.0
21,22,I agree,"I am not primarily a developer, but I write co...",25-34 years old,"Employed, full-time",Remote,Hobby,"Professional degree (JD, MD, Ph.D, Ed.D, etc.)",Colleague;Hackathons (virtual or in-person);On...,Formal documentation provided by the owner of ...,...,1-2 times a week,3-5 times a week,1-2 times a week,15-30 minutes a day,Less than 15 minutes a day,None of these,NaN,Appropriate in length,Easy,85672.0
23,24,I agree,"I am not primarily a developer, but I write co...",25-34 years old,"Employed, full-time",Remote,I don’t code outside of work,"Professional degree (JD, MD, Ph.D, Ed.D, etc.)",Colleague;On the job training;Other online res...,Formal documentation provided by the owner of ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,69608.0
32,33,I agree,I am a developer by profession,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Secondary school (e.g. American high school, G...",Books / Physical media;Colleague;On the job tr...,Formal documentation provided by the owner of ...,...,1-2 times a week,1-2 times a week,Never,15-30 minutes a day,15-30 minutes a day,DevOps function;Microservices;Observability to...,Other,Appropriate in length,Easy,53545.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89088,89089,I agree,I am a developer by profession,35-44 years old,"Employed, full-time","Hybrid (some remote, some in-person)",I don’t code outside of work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Formal documentation provided by the owner of ...,...,1-2 times a week,1-2 times a week,1-2 times a week,60-120 minutes a day,30-60 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Too long,Neither easy nor difficult,101735.0
89107,89108,I agree,I am a developer by profession,35-44 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Bootstrapping a business,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Hackathons (virtual or ...,NaN,...,1-2 times a week,3-5 times a week,1-2 times a week,15-30 minutes a day,Over 120 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Appropriate in length,Easy,NaN
89125,89126,I agree,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,Professional development or self-paced learnin...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Books;Video-based Online Courses;Stack

In [65]:
df_new4.to_csv('/content/drive/My Drive/RediSchool-salarypredictor/survey_results_v3.csv')
